# Hackerearth Exhibit A(rt)

Hackathon for predicting the cost of shipping an art piece.

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling

#Training data file
train_file = "./dataset/train.csv"
test_file = "./dataset/test.csv"

## Data Specification

| Column name | Description |
| --- | --- |
| Customer Id | Represents the unique identification number of the customers |
| Artist Name |Represents the name of the artist |
| Artist Reputation	| Represents the reputation of an artist in the market (the greater the reputation value, the higher the reputation of the artist in the market)
| Height | Represents the height of the sculpture |
| Width	| Represents the width of the sculpture |
| Weight | Represents the weight of the sculpture |
| Material | Represents the material that the sculpture is made of |
| Price Of Sculpture | Represents the price of the sculpture |
| Base Shipping Price | Represents the base price for shipping a sculpture |
| International | Represents whether the shipping is international |
| Express Shipment | Represents whether the shipping was in the express (fast) mode |
| Installation Included | Represents whether the order had installation included in the purchase of the sculpture |
| Transport | Represents the mode of transport of the order |
| Fragile | Represents whether the order is fragile |
| Customer Information | Represents details about a customer |
| Remote Location | Represents whether the customer resides in a remote location |
| Scheduled Date | Represents the date when the order was placed |
| Delivery Date | Represents the date of delivery of the order |
| Customer Location | Represents the location of the customer |
| Cost | Represents the cost of the order |

## Load the data in a pandas.Dataframe

In [2]:
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
#train_df.profile_report()

In [3]:
train_df.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",-159.23


In [4]:
#Separate the target in dataframe
y_train = train_df["Cost"].abs()
x_train = train_df.drop(["Cost"], axis=1)
x_test = test_df
missing_values_count = x_train.isnull().sum()
missing_values_count

Customer Id                 0
Artist Name                 0
Artist Reputation         750
Height                    375
Width                     584
Weight                    587
Material                  764
Price Of Sculpture          0
Base Shipping Price         0
International               0
Express Shipment            0
Installation Included       0
Transport                1392
Fragile                     0
Customer Information        0
Remote Location           771
Scheduled Date              0
Delivery Date               0
Customer Location           0
dtype: int64

## Inspect the data
### Observations:
- CustomerIds are 100% unique, might be representing transaction ID(candidate for removal)
- Customer Locations are 100% unique(candidate for removal since distance cannot be dervided from currently provided data)
- Cost is highly skewed(Target to be predicted)
- Price of sculpture is highly skewed
- Height, Width, Weight and Material might have correlation(Try to use PCA and another regressor to impute missing values)
- Transportation has a lot of missing information
### Columns with missing data
- Artist Reputation
- Height
- Width
- Weight
- Material
- Transport
- Remote Location

## Approach 1: Straight forward, just use features that affected shipping cost based on experience
Remove probably not useful columns
CustomerIds, Customer Location, Material, Artist Name, Artist Reputation, Scheduled Date, Delivery Date
### Note
Consider adding again, Artist reputation in later attempt

In [5]:
drop_columns = ["Customer Id", "Customer Location", "Material", "Artist Name", "Artist Reputation", "Scheduled Date", "Delivery Date"]
X_train_dropped = x_train.drop(drop_columns, axis=1)
X_test_dropped = x_test.drop(drop_columns, axis=1)
label = X_train_dropped.dtypes == "object"
label_enc_cols = list(label[label].index)

### Fill missing values of Transport

In [6]:
replacement = X_train_dropped["Transport"].mode()[0]
X_train_dropped["Transport"] = X_train_dropped["Transport"].fillna(replacement)
replacement = X_train_dropped["Remote Location"].mode()[0]
X_train_dropped["Remote Location"] = X_train_dropped["Remote Location"].fillna(replacement)
replacement = X_test_dropped["Transport"].mode()[0]
X_test_dropped["Transport"] = X_test_dropped["Transport"].fillna(replacement)
replacement = X_test_dropped["Remote Location"].mode()[0]
X_test_dropped["Remote Location"] = X_test_dropped["Remote Location"].fillna(replacement)

### Label Encode

In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in label_enc_cols:
    X_train_dropped[col] = label_encoder.fit_transform(X_train_dropped[col])
    X_test_dropped[col] = label_encoder.fit_transform(X_test_dropped[col])

In [8]:
X_train_dropped.head()

,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location
0,17.0,6.0,4128.0,13.91,16.27,1,1,0,0,0,1,0
1,3.0,3.0,61.0,6.83,15.00,0,0,0,1,0,1,0
2,8.0,5.0,237.0,4.96,21.18,0,0,0,1,1,1,1
3,9.0,NaN,NaN,5.81,16.31,0,0,0,1,0,0,1
4,17.0,6.0,324.0,3.18,11.94,1,1,1,0,0,1,0


## Impute missing values

In [9]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent")
X_imputed = pd.DataFrame(imputer.fit_transform(X_train_dropped), columns=X_train_dropped.columns)

## Scale the data

In [10]:
from sklearn.preprocessing import MinMaxScaler
feat_to_scale = ["Height", "Width", "Weight","Price Of Sculpture", "Base Shipping Price"]
X_scaled = X_imputed
scaled_features = MinMaxScaler().fit_transform(X_scaled[feat_to_scale])
#Remove unscaled columns from original DF
scaled_feature_df = pd.DataFrame(scaled_features, columns=feat_to_scale)
scaled_feature_df.head()
for col in feat_to_scale:
    X_scaled[col] = scaled_feature_df[col]
    
X_scaled.head()
y_train = np.log1p(y_train)

In [11]:
from sklearn.model_selection import train_test_split
# Split the currently available training set for evaluation
train_feat, test_feat, cost_train, cost_test = train_test_split(
    X_scaled.to_numpy(),
    y_train.to_numpy(),
    train_size=0.8,
    random_state=0
    )


## Try AdaBoost Regressor

In [12]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(random_state=0, n_estimators=150, learning_rate=0.0001, loss="exponential")

In [13]:
ada.fit(train_feat, cost_train)
print(ada.score(test_feat, cost_test))

0.8606118766716299


## RandomForest Regressor

In [14]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=150,random_state=42)

In [15]:
rfr.fit(train_feat, cost_train)
print(rfr.score(test_feat, cost_test))

0.8892530789905283


## Linear Regressor

In [16]:
from sklearn.linear_model import LinearRegression
lnr = LinearRegression(normalize=True)

In [17]:
lnr.fit(train_feat, cost_train)
print(lnr.score(test_feat, cost_test))

0.5699499216959859


## Ridge Regression

In [18]:
from sklearn.linear_model import Ridge
rdg = Ridge(alpha=0.1, normalize=True)
rdg.fit(train_feat, cost_train)
print(rdg.score(test_feat, cost_test))

0.5665261893013835


## Kernel Ridge

In [19]:
from sklearn.kernel_ridge import KernelRidge
krn = KernelRidge(alpha=1000, degree=12)
krn.fit(train_feat, cost_train)
print(krn.score(test_feat, cost_test))

-0.6112040000880068


## GradientBoosting

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
grd = GradientBoostingRegressor(random_state=0)
grd.fit(train_feat, cost_train)
print(grd.score(test_feat, cost_test))

0.8896623379793243


## Approach 1 result
- Gradient Boosting performed better than other models using the present data

## Approach 2: Generate additional features from present data

In [21]:
# Original Data
x_modified = train_df
x_modified["Scheduled Date"] = pd.to_datetime(x_modified["Scheduled Date"], format="%m/%d/%y")
x_modified["Delivery Date"] = pd.to_datetime(x_modified["Delivery Date"], format="%m/%d/%y")
x_modified.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,2015-06-07,2015-06-03,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,2017-03-06,2017-03-05,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,2015-03-09,2015-03-08,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,2015-05-24,2015-05-20,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,2016-12-18,2016-12-14,"Aprilside, PA 52793",-159.23


### Create new feature from difference of scheduled date and delivery date

In [22]:
x_modified["ScheduleDiff"] = (x_modified["Delivery Date"]-x_modified["Scheduled Date"]).map(lambda x: str(x).split()[0])
x_modified["ScheduleDiff"] = pd.to_numeric(x_modified["ScheduleDiff"])

## Extract state from customer location

In [23]:
x_modified["State"] = x_modified["Customer Location"].map(lambda x:str(x).split()[-2])

In [24]:
#x_modified.profile_report()

In [25]:
y_log = y_train